In [11]:
import pandas as pd

In [12]:
def read_excel_multisheet(filename: str) -> pd.DataFrame:
    workbook = pd.ExcelFile(filename)
    sheets = workbook.sheet_names
    df = pd.concat([pd.read_excel(workbook, sheet_name=s, index_col="Unnamed: 0").sort_index()
                    .assign(source=s) for s in sheets])
    return df

В data_dict были дубликаты. Но чтобы ничего не ломать, его мы трогать не будем, а удалим дубликаты из сформированных на основе его датасетов:

In [ ]:
fact_scores = pd.read_excel("../data_output_table/distractor_metrics/DistractorFactScores1.xlsx", engine="openpyxl")
gemini_scores = pd.read_excel("../data_output_table/auto_assess_distractors/gemini_results.xlsx", engine="openpyxl", index_col="Unnamed: 0")

In [14]:
fact_scores.shape, fact_scores["source"].value_counts()

((1561, 14),
 source
 BartDG_PM           167
 BartDG_ANPM         167
 BartDG              166
 ChatGPT4o           166
 true_distractors    166
 Deepseek            165
 RuRace_T5           160
 MuSeRC_GPT3         159
 RuRace_GPT3         156
 MuSeRC_T5            89
 Name: count, dtype: int64)

In [15]:
gemini_scores.shape, gemini_scores["source"].value_counts()

((1597, 5),
 source
 BartDG_PM           167
 BartDG_ANPM         167
 ChatGPT4o           167
 BartDG              166
 true_distractors    166
 RuRace_T5           165
 Deepseek            165
 MuSeRC_GPT3         159
 RuRace_GPT3         156
 MuSeRC_T5           119
 Name: count, dtype: int64)

In [16]:
fact_scores = fact_scores.drop_duplicates(subset=[
    "source", "reading_text", "distractor"
])
gemini_scores = gemini_scores.drop_duplicates(subset=[
    "source", "reading_text", "question", "distractor"
])

In [17]:
fact_scores.shape, fact_scores["source"].value_counts()

((1561, 14),
 source
 BartDG_PM           167
 BartDG_ANPM         167
 BartDG              166
 ChatGPT4o           166
 true_distractors    166
 Deepseek            165
 RuRace_T5           160
 MuSeRC_GPT3         159
 RuRace_GPT3         156
 MuSeRC_T5            89
 Name: count, dtype: int64)

In [18]:
gemini_scores.shape, gemini_scores["source"].value_counts()

((1561, 5),
 source
 BartDG_PM           167
 BartDG_ANPM         167
 BartDG              166
 ChatGPT4o           166
 true_distractors    166
 Deepseek            165
 RuRace_T5           160
 MuSeRC_GPT3         159
 RuRace_GPT3         156
 MuSeRC_T5            89
 Name: count, dtype: int64)

In [19]:
fact_scores.head()

,Unnamed: 0,source,vso_intersec_ind,vs_intersec_ind,vs_passivized_intersec_ind,vso_intersec,vs_intersec,vs_passivized_intersec,noun_intersec,propn_intersec,noun_intersec_ind,propn_intersec_ind,reading_text,distractor
0,0,BartDG,0,0,0,set(),set(),set(),{'родина'},set(),1,0,На задворках нашего села стояло на сваях длин...,Автор очень любил свою родину.
1,1,BartDG,0,0,0,set(),set(),set(),"{'скрипка', 'родина'}",set(),1,0,На задворках нашего села стояло на сваях длин...,Автор часто играл на скрипке на своей родине.
2,2,BartDG,0,0,0,set(),set(),set(),"{'война', 'жизнь'}",set(),1,0,На задворках нашего села стояло на сваях длин...,Соотечественник автора спас жизнь своему сыну ...
3,3,BartDG,0,0,0,set(),set(),set(),set(),set(),0,0,"Мы ехали берегом Лены на юг, а зима догоняла ...",Оба козла были горными козлами.
4,4,BartDG,0,0,0,set(),set(),set(),{'коза'},set(),1,0,"Мы ехали берегом Лены на юг, а зима догоняла ...","Старая коза, маленькая, была готова снова прыг..."


In [20]:
gemini_scores.head()

,source,reading_text,question,distractor,gemini_guess
0,BartDG,На задворках нашего села стояло на сваях длин...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Автор очень любил свою родину.,1
1,BartDG,На задворках нашего села стояло на сваях длин...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Автор часто играл на скрипке на своей родине.,0
2,BartDG,На задворках нашего села стояло на сваях длин...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Соотечественник автора спас жизнь своему сыну ...,0
3,BartDG,"Мы ехали берегом Лены на юг, а зима догоняла ...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Оба козла были горными козлами.,0
4,BartDG,"Мы ехали берегом Лены на юг, а зима догоняла ...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,"Старая коза, маленькая, была готова снова прыг...",0


In [21]:
scores_combined = pd.merge(
    gemini_scores[["source", "reading_text", "question", "distractor", "gemini_guess"]],
    fact_scores[
        ["source", "reading_text", "distractor",
         "vso_intersec_ind", "vs_intersec_ind", "vs_passivized_intersec_ind",
         "noun_intersec_ind", "propn_intersec_ind"
        ]
    ], on=["source", "reading_text", "distractor"]
)

In [ ]:
scores_combined.to_excel("../data_for_comparison/corr/scores_combined.xlsx")

In [23]:
q_cols = ["gemini_guess", "vso_intersec_ind", "vs_intersec_ind", "vs_passivized_intersec_ind", "noun_intersec_ind", "propn_intersec_ind"]

In [24]:
source_totals = scores_combined.groupby("source")[q_cols].mean()
table2 = scores_combined.groupby(["source", "question"])[q_cols].mean()

for source in scores_combined["source"].unique():
    table2.loc[(source, "total"), :] = source_totals.loc[source]
    table2.loc[(source, "total"), "D/Q"] = len(scores_combined[scores_combined["source"] == source]) / 55
    table2.loc[(source, "Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?"), "D/Q"] = len(
        scores_combined[(scores_combined["source"] == source) & (scores_combined["question"] == "Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?")]
    ) / 35
    table2.loc[(source, "Какое высказывание СООТВЕТСТВУЕТ тексту?"), "D/Q"] = len(
        scores_combined[(scores_combined["source"] == source) & (scores_combined["question"] == "Какое высказывание СООТВЕТСТВУЕТ тексту?")]
    ) / 20

table2 = table2.sort_index()

table2

gemini_guess  \
source           question                                                    
BartDG           total                                            0.500000   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.254717   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.933333   
BartDG_ANPM      total                                            0.443114   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.188679   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.885246   
BartDG_PM        total                                            0.461078   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.186916   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.950000   
ChatGPT4o        total                                            0.536145   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.301887   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.950000   
Deepseek         total                                            0.460606   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.171429   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.966667   
MuSeRC_GPT3      total                                            0.383648   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.147059   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.807018   
MuSeRC_T5        total                                            0.426966   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.230769   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.702703   
RuRace_GPT3      total                                            0.391026   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.100000   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.910714   
RuRace_T5        total                                            0.456250   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.237624   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.830508   
true_distractors total                                            0.957831   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?      0.962264   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?         0.950000   

                                                              vso_intersec_ind  \
source           question                                                        
BartDG           total                                                0.024096   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?          0.037736   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?             0.000000   
BartDG_ANPM      total                                                0.011976   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?          0.009434   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?             0.016393   
BartDG_PM        total                                                0.000000   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?          0.000000   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?             0.000000   
ChatGPT4o        total                                                0.006024   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?          0.000000   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?             0.016667   
Deepseek         total                                                0.006061   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?          0.000000   
                 Какое высказывание СООТВЕТСТВУЕТ тексту?             0.016667   
MuSeRC_GPT3      total                                                0.012579   
                 Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?          0.009804   
       

In [25]:
def map_question_texts(s: str) -> str:
    outp = s
    if s == "Какое высказывание СООТВЕТСТВУЕТ тексту?":
        outp = "СООТВ"
    elif s == "Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?":
        outp = "НЕ СООТВ"
    return outp

table2 = table2.reset_index()
table2["question"] = table2["question"].map(map_question_texts)
table2 = table2.set_index(["source", "question"])

In [26]:
table2

gemini_guess  vso_intersec_ind  vs_intersec_ind  \
source           question                                                    
BartDG           total         0.500000          0.024096         0.060241   
                 НЕ СООТВ      0.254717          0.037736         0.084906   
                 СООТВ         0.933333          0.000000         0.016667   
BartDG_ANPM      total         0.443114          0.011976         0.041916   
                 НЕ СООТВ      0.188679          0.009434         0.047170   
                 СООТВ         0.885246          0.016393         0.032787   
BartDG_PM        total         0.461078          0.000000         0.011976   
                 НЕ СООТВ      0.186916          0.000000         0.018692   
                 СООТВ         0.950000          0.000000         0.000000   
ChatGPT4o        total         0.536145          0.006024         0.042169   
                 НЕ СООТВ      0.301887          0.000000         0.047170   
                 СООТВ         0.950000          0.016667         0.033333   
Deepseek         total         0.460606          0.006061         0.048485   
                 НЕ СООТВ      0.171429          0.000000         0.019048   
                 СООТВ         0.966667          0.016667         0.100000   
MuSeRC_GPT3      total         0.383648          0.012579         0.018868   
                 НЕ СООТВ      0.147059          0.009804         0.019608   
                 СООТВ         0.807018          0.017544         0.017544   
MuSeRC_T5        total         0.426966          0.056180         0.067416   
                 НЕ СООТВ      0.230769          0.038462         0.038462   
                 СООТВ         0.702703          0.081081         0.108108   
RuRace_GPT3      total         0.391026          0.019231         0.038462   
                 НЕ СООТВ      0.100000          0.020000         0.030000   
                 СООТВ         0.910714          0.017857         0.053571   
RuRace_T5        total         0.456250          0.043750         0.100000   
                 НЕ СООТВ      0.237624          0.059406         0.108911   
                 СООТВ         0.830508          0.016949         0.084746   
true_distractors total         0.957831          0.024096         0.054217   
                 НЕ СООТВ      0.962264          0.028302         0.066038   
                 СООТВ         0.950000          0.016667         0.033333   

                           vs_passivized_intersec_ind  noun_intersec_ind  \
source           question                                                  
BartDG           total                       0.006024           0.831325   
                 НЕ СООТВ                    0.009434           0.867925   
                 СООТВ                       0.000000           0.766667   
BartDG_ANPM      total                       0.000000           0.862275   
                 НЕ СООТВ                    0.000000           0.905660   
                 СООТВ                       0.000000           0.786885   
BartDG_PM        total                       0.005988           0.808383   
                 НЕ СООТВ                    0.009346           0.906542   
                 СООТВ                       0.000000           0.633333   
ChatGPT4o        total                       0.000000           0.873494   
                 НЕ СООТВ                    0.000000           0.877358   
                 СООТВ                       0.000000           0.866667   
Deepseek         total                       0.000000           0.957576   
                 НЕ СООТВ                    0.000000           0.971429   
                 СООТВ                       0.000000           0.933333   
MuSeRC_GPT3      total                       0.000000           0.389937   
                 НЕ СООТВ                    0.000000           0.411765   
                 СООТВ                       0.000000           0.350877   
MuSeRC_T5 

In [27]:
table2 = table2.drop(["vs_passivized_intersec_ind"], axis="columns")

In [28]:
table2 = table2.rename(columns={"vso_intersec_ind": "FullNPMatch", "vs_intersec_ind": "SubjNPMatch", "noun_intersec_ind": "NounMatch", "propn_intersec_ind": "PropnMatch"})
table2

gemini_guess  FullNPMatch  SubjNPMatch  NounMatch  \
source           question                                                      
BartDG           total         0.500000     0.024096     0.060241   0.831325   
                 НЕ СООТВ      0.254717     0.037736     0.084906   0.867925   
                 СООТВ         0.933333     0.000000     0.016667   0.766667   
BartDG_ANPM      total         0.443114     0.011976     0.041916   0.862275   
                 НЕ СООТВ      0.188679     0.009434     0.047170   0.905660   
                 СООТВ         0.885246     0.016393     0.032787   0.786885   
BartDG_PM        total         0.461078     0.000000     0.011976   0.808383   
                 НЕ СООТВ      0.186916     0.000000     0.018692   0.906542   
                 СООТВ         0.950000     0.000000     0.000000   0.633333   
ChatGPT4o        total         0.536145     0.006024     0.042169   0.873494   
                 НЕ СООТВ      0.301887     0.000000     0.047170   0.877358   
                 СООТВ         0.950000     0.016667     0.033333   0.866667   
Deepseek         total         0.460606     0.006061     0.048485   0.957576   
                 НЕ СООТВ      0.171429     0.000000     0.019048   0.971429   
                 СООТВ         0.966667     0.016667     0.100000   0.933333   
MuSeRC_GPT3      total         0.383648     0.012579     0.018868   0.389937   
                 НЕ СООТВ      0.147059     0.009804     0.019608   0.411765   
                 СООТВ         0.807018     0.017544     0.017544   0.350877   
MuSeRC_T5        total         0.426966     0.056180     0.067416   0.662921   
                 НЕ СООТВ      0.230769     0.038462     0.038462   0.673077   
                 СООТВ         0.702703     0.081081     0.108108   0.648649   
RuRace_GPT3      total         0.391026     0.019231     0.038462   0.775641   
                 НЕ СООТВ      0.100000     0.020000     0.030000   0.800000   
                 СООТВ         0.910714     0.017857     0.053571   0.732143   
RuRace_T5        total         0.456250     0.043750     0.100000   0.900000   
                 НЕ СООТВ      0.237624     0.059406     0.108911   0.910891   
                 СООТВ         0.830508     0.016949     0.084746   0.881356   
true_distractors total         0.957831     0.024096     0.054217   0.909639   
                 НЕ СООТВ      0.962264     0.028302     0.066038   0.915094   
                 СООТВ         0.950000     0.016667     0.033333   0.900000   

                           PropnMatch       D/Q  
source           question                        
BartDG           total       0.228916  3.018182  
                 НЕ СООТВ    0.150943  3.028571  
                 СООТВ       0.366667  3.000000  
BartDG_ANPM      total       0.143713  3.036364  
                 НЕ СООТВ    0.084906  3.028571  
                 СООТВ       0.245902  3.050000  
BartDG_PM        total       0.137725  3.036364  
                 НЕ СООТВ    0.093458  3.057143  
                 СООТВ       0.216667  3.000000  
ChatGPT4o        total       0.331325  3.018182  
                 НЕ СООТВ    0.311321  3.028571  
                 СООТВ       0.366667  3.000000  
Deepseek         total       0.345455  3.000000  
                 НЕ СООТВ    0.304762  3.000000  
                 СООТВ       0.416667  3.000000  
MuSeRC_GPT3      total       0.069182  2.890909  
                 НЕ СООТВ    0.049020  2.914286  
                 СООТВ       0.105263  2.850000  
MuSeRC_T5        total       0.292135  1.618182  
                 НЕ СООТВ    0.269231  1.485714  
                 СООТВ       0.324324  1.850000  
RuRace_GPT3      total       0.256410  2.836364  
                 НЕ СООТВ    0.190000  2.857143  
                 СООТВ       0.375000  2.800000  
RuRace_T5        total       0.337500  2.909091  
                 НЕ СООТВ    0.287129  2.885714  
                 СООТВ       0.423729  2.950000  
true_distractors

In [29]:
table2["gemini_guess"] = table2["gemini_guess"] * 100
table2["FullNPMatch"] = table2["FullNPMatch"] * 100
table2["SubjNPMatch"] = table2["SubjNPMatch"] * 100
table2["NounMatch"] = table2["NounMatch"] * 100
table2["PropnMatch"] = table2["PropnMatch"] * 100

In [ ]:
table2.to_excel("../data_output_table/tables_for_manuscript/Table2-1.xlsx", float_format="%.2f")